## IMPORT LIBRARIES

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import re

import warnings
warnings.filterwarnings('ignore')

## OVERVIEW

In [2]:
df = pd.read_csv('dataset\movie.csv')

In [3]:
df.head(3)

,user_id,event_id,transaction_date,event_type,organization_type,event_category,user_type,user_location,gender,age_when_register,price,total_ticket_sold
0,50bd7b56b494e843cf8cc86deba1746b5dcb3622871c3c...,3757175fd0193ff5f0610a51c0c66bc2f23a027c76b0f7...,2019-01-01 05:51:06,Movie,Cinemas,Movies or Cinemas,Registered,Bandung,female,17.0,50000,3
1,5ab517007fc2fa2256ccb5179cf2f85fb6333c0b97e78f...,ce3ef49344353815e0e6bb9be1e94cae9e3dcf19e21931...,2019-01-03 06:24:18,Movie,Cinemas,Movies or Cinemas,Registered,Depok,female,21.0,35000,2
2,b9c12d8ea302adcf5c1583367702dfeff02c00bb3a247b...,b8e87ba403345ef34659cfce029a68f73ec6f89eee59e2...,2019-01-03 07:55:14,Movie,Cinemas,Movies or Cinemas,Registered,Jakarta,male,25.0,50000,3


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15138 entries, 0 to 15137
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   user_id            15138 non-null  object 
 1   event_id           15138 non-null  object 
 2   transaction_date   15138 non-null  object 
 3   event_type         15138 non-null  object 
 4   organization_type  15138 non-null  object 
 5   event_category     15138 non-null  object 
 6   user_type          15138 non-null  object 
 7   user_location      10457 non-null  object 
 8   gender             15115 non-null  object 
 9   age_when_register  12256 non-null  float64
 10  price              15138 non-null  int64  
 11  total_ticket_sold  15138 non-null  int64  
dtypes: float64(1), int64(2), object(9)
memory usage: 1.4+ MB


In [5]:
pd.DataFrame({'dataFeatures' : df.columns, 'dataType' : df.dtypes.values, 
              'null' : [df[i].isna().sum() for i in df.columns],
              'nullPct' : [((df[i].isna().sum()/len(df[i]))*100).round(1) for i in df.columns],
              'Nunique' : [df[i].nunique() for i in df.columns],
              'uniqueSample' : [list(pd.Series(df[i].unique()).sample()) for i in df.columns]}).reset_index(drop = True)

,dataFeatures,dataType,null,nullPct,Nunique,uniqueSample
0,user_id,object,0,0.0,5038,[7a8aed27e48f168ae7dd929b4ba27d825f59ae53281be...
1,event_id,object,0,0.0,5289,[349c441f5562531a58593751f2a613d76ae0c864d9448...
2,transaction_date,object,0,0.0,14945,[2019-02-27 06:46:47]
3,event_type,object,0,0.0,1,[Movie]
4,organization_type,object,0,0.0,1,[Cinemas]
5,event_category,object,0,0.0,1,[Movies or Cinemas]
6,user_type,object,0,0.0,2,[Guest]
7,user_location,object,4681,30.9,30,[Cirebon]
8,gender,object,23,0.2,3,[nan]
9,age_when_register,float64,2882,19.0,60,[62.0]


In [6]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
age_when_register,12256.0,27.548548,10.576091,9.0,21.0,24.0,30.0,90.0
price,15138.0,40710.959176,14850.901880,5000.0,35000.0,35000.0,45000.0,500000.0
total_ticket_sold,15138.0,1.672678,0.947632,1.0,1.0,2.0,2.0,8.0


In [7]:
df.describe(include = 'object').T

,count,unique,top,freq
user_id,15138,5038,fd1d86e37e79a7f3412818a79f22a3fccb52ced852664e...,555
event_id,15138,5289,c71952786040a4b56ac78f3bd22b8608e4b32da54b457f...,206
transaction_date,15138,14945,2019-11-07 07:12:20,4
event_type,15138,1,Movie,15138
organization_type,15138,1,Cinemas,15138
event_category,15138,1,Movies or Cinemas,15138
user_type,15138,2,Registered,12306
user_location,10457,30,Yogyakarta,4205
gender,15115,3,female,6576


## PREPROCESSING

> Let's start from **user location**.

In [8]:
df[df['user_location'].isna()]

,user_id,event_id,transaction_date,event_type,organization_type,event_category,user_type,user_location,gender,age_when_register,price,total_ticket_sold
46,c43bc48c9bdb9f609879b41bf36ab18a51641620994b2c...,ecb2b8f3697aafad4ee4acf5ea232a5aa5f767877237db...,2019-02-22 03:21:39,Movie,Cinemas,Movies or Cinemas,Guest,NaN,none,NaN,40000,2
89,367b471b3e1707de4303ee0525a54d212f93a2177897a1...,40d58c0fd8b2249a8759418a61f7fb97bfd857edfae4d9...,2019-02-23 05:00:30,Movie,Cinemas,Movies or Cinemas,Guest,NaN,none,NaN,75000,2
94,97f0e8938bbfe3e6245fee82693ae69577545ac12c4f90...,97ee1568136dde752d5d6d9f1c64d1e368d3a060246732...,2019-02-23 09:06:19,Movie,Cinemas,Movies or Cinemas,Guest,NaN,none,NaN,45000,2
98,ba87885bc2cce3afb0915f6d4bee29f3f7926ed03c75bf...,d344e1c3d0313efbdffc363d91e8342dcb56ca199ca290...,2019-02-23 10:11:00,Movie,Cinemas,Movies or Cinemas,Guest,NaN,none,NaN,50000,2
100,a0145331389e0662f4c99950648bad94da75583d354377...,a1de5e094330672b35dd2f89827649bac9f64ecd5777de...,2019-02-23 11:04:25,Movie,Cinemas,Movies or Cinemas,Guest,NaN,NaN,NaN,45000,2
...,...,...,...,...,...,...,...,...,...,...,...,...
15130,fc1bdbf6277f8fa7cac344973a0835efeb04dfca5a95c0...,e13eff18a2faa1640ebfdd6af5b2f2fd11dcfe1d7fd58f...,2020-03-06 10:53:16,Movie,Cinemas,Movies or Cinemas,Guest,NaN,none,NaN,55000,2
15131,fc1bdbf6277f8fa7cac344973a0835efeb04dfca5a95c0...,e13eff18a2faa1640ebfdd6af5b2f2fd11dcfe1d7fd58f...,2020-03-06 10:57:19,Movie,Cinemas,Movies or Cinemas,Guest,NaN,none,NaN,55000,2
15134,f020e02f914012b2642463c4f26500cf63f98fc7e92222...,53fc7be9f584c15718528862984e563568fac7cf72183d...,2020-03-13 12:37:30,Movie,Cinemas,Movies or Cinemas,Registered,NaN,female,39.0,62500,2
15135,d0f508f59f2d9450cc503a0f93ba7e26c670636953a995...,f283497b15d2cefc4507a5d65f90a4bc5effcab4e83ef4...,2020-03-13 12:43:15,Movie,Cinemas,Movies or Cinemas,Registered,NaN,female,39.0,47500,2


In [9]:
df[df['user_location'].isna()]['user_type'].value_counts()

Guest         2830
Registered    1851
Name: user_type, dtype: int64

In [10]:
loc_regist_isna = df[(df['user_location'].isna()) & (df['user_type']=='Registered')]
loc_regist_notna = df[(df['user_location'].notna()) & (df['user_type']=='Registered')]

In [11]:
loc_regist_isna.head()

,user_id,event_id,transaction_date,event_type,organization_type,event_category,user_type,user_location,gender,age_when_register,price,total_ticket_sold
1042,ae4d632899331f334565b9b81629f0395abcab838bb91d...,a8955367f9abec178f005bdad41c1988f70d7b48480539...,2019-05-08 05:56:49,Movie,Cinemas,Movies or Cinemas,Registered,NaN,male,19.0,42500,2
1059,ae4d632899331f334565b9b81629f0395abcab838bb91d...,deb7319effac93639b51eadd87fe1ae35fe1085b32cc55...,2019-05-16 05:42:22,Movie,Cinemas,Movies or Cinemas,Registered,NaN,male,19.0,42500,2
3136,7d55d8e8a3a007bf5ccf405ec1cd5e87483fecfc07b4a9...,2f061efe793c19c7c4d183d0ef9fb0030cac9463e66446...,2019-11-07 06:27:30,Movie,Cinemas,Movies or Cinemas,Registered,NaN,male,21.0,35000,1
5312,756f458f44b4c047ed31e9346a2e804d13488cd1badd82...,c41d40719d214a3a52f56c4e8d252317814c0565e6707c...,2019-11-08 09:10:50,Movie,Cinemas,Movies or Cinemas,Registered,NaN,female,22.0,35000,1
5317,756f458f44b4c047ed31e9346a2e804d13488cd1badd82...,4f22a24f4a48e420394f4a70558815d90013d4cd8409e2...,2019-11-08 09:27:25,Movie,Cinemas,Movies or Cinemas,Registered,NaN,female,22.0,35000,1


In [12]:
loc_regist_notna.groupby('event_id')['user_location'].value_counts()

event_id                                                          user_location
000c00e8322eb941989528878f01f6add536a8605fbc0ce7b51d435257950ae0  Bandung          1
001ddd446a41b0fc4578b077b57f7c51faf3fe0d420eaa4ac256f4427c5fb765  Solo             1
0021a23a1ad6542f63317318470cf33a00d9bb97fc28c5bee08ab94ceb898cb1  Jakarta          1
00529ed280a461499121dfe398548b5f8abe8931cd8370f9480a0cc16d3fea51  Karawang         1
0052b33771d32eb1cb5585f030e6ca9b9b937ada33c26786f28258cc48fd8c9b  Purwakarta       1
                                                                                  ..
ffbdae7028c8079b6f6d9e643cdfc6fbd2b532649f759e6f6186a2a76e1f6fe8  Medan            1
ffcb2a5a0c81b060ec7718c995054191462b77ad36e0525591a92d6fc289144f  Bandung          1
ffd45f9312cc2c9758c74def3d4181451d750dc3141e5679ff2686eb7acf128d  Tangerang        1
fff6b720244784039b5c481df195b63b0def753a5011bdee44f3cf535f5a5220  Blitar           1
fffe648ef4d74ac326dbb995d3c282960ec78c26aacea5cfb6193e96560bf039  Peka

In [13]:
df[df['event_id']=='0021a23a1ad6542f63317318470cf33a00d9bb97fc28c5bee08ab94ceb898cb1']

,user_id,event_id,transaction_date,event_type,organization_type,event_category,user_type,user_location,gender,age_when_register,price,total_ticket_sold
1023,97a2b0837b6bd4201cff410dbd4d242bb8992d24c305b0...,0021a23a1ad6542f63317318470cf33a00d9bb97fc28c5...,2019-04-30 07:37:32,Movie,Cinemas,Movies or Cinemas,Registered,Jakarta,male,33.0,75000,4


In [14]:
loc_regist_isna[loc_regist_isna['user_location'].isin(loc_regist_notna['user_location'])]

,user_id,event_id,transaction_date,event_type,organization_type,event_category,user_type,user_location,gender,age_when_register,price,total_ticket_sold


> I think it's rather difficult to fill user location with no lead whatsoever. Therefore I'll fill these null values as **Unknown**.

In [15]:
df['user_location'].fillna('Unknown', inplace=True)

In [16]:
df.isna().sum()

user_id                 0
event_id                0
transaction_date        0
event_type              0
organization_type       0
event_category          0
user_type               0
user_location           0
gender                 23
age_when_register    2882
price                   0
total_ticket_sold       0
dtype: int64

> Now moving to **gender**.

In [17]:
df[df['gender'].isna()]

,user_id,event_id,transaction_date,event_type,organization_type,event_category,user_type,user_location,gender,age_when_register,price,total_ticket_sold
100,a0145331389e0662f4c99950648bad94da75583d354377...,a1de5e094330672b35dd2f89827649bac9f64ecd5777de...,2019-02-23 11:04:25,Movie,Cinemas,Movies or Cinemas,Guest,Unknown,NaN,NaN,45000,2
858,fed7fa12df6ff811b748e624f7177ad8d4ed4c2a75b49e...,7c1be0c107214fce0b35d09b4647b588108aa2d25c6cf7...,2019-04-21 09:49:41,Movie,Cinemas,Movies or Cinemas,Guest,Unknown,NaN,NaN,75000,2
7594,ae72051054c987c1aca286cd33d9081d34f6ba284693ba...,40affd7b6039825eefbddaf2557d5358abfcb2d3e4a72d...,2019-11-21 08:06:15,Movie,Cinemas,Movies or Cinemas,Guest,Unknown,NaN,NaN,35000,1
9773,78310f1b44b7e2edd0e61b0cc54de97e58ab35df437f2a...,74d1b71fa7a7a7c15f33371cdd80d1ed0462a4c21d6965...,2019-04-20 08:33:27,Movie,Cinemas,Movies or Cinemas,Guest,Unknown,NaN,NaN,50000,1
9832,78310f1b44b7e2edd0e61b0cc54de97e58ab35df437f2a...,658866c9cd70c8ad55d34d5313913474227d67d076de7b...,2019-04-21 08:12:38,Movie,Cinemas,Movies or Cinemas,Guest,Unknown,NaN,NaN,50000,1
13726,ca7e910f39fa0c5cfc13ce6da2908c5dc44383cd0e5dfd...,1532bb3b327ee704d022769c499067b85d516a3a015e43...,2019-11-08 21:54:35,Movie,Cinemas,Movies or Cinemas,Guest,Unknown,NaN,NaN,35000,4
13728,ca7e910f39fa0c5cfc13ce6da2908c5dc44383cd0e5dfd...,c41d40719d214a3a52f56c4e8d252317814c0565e6707c...,2019-11-08 22:04:35,Movie,Cinemas,Movies or Cinemas,Guest,Unknown,NaN,NaN,35000,4
13730,ca7e910f39fa0c5cfc13ce6da2908c5dc44383cd0e5dfd...,99a4090c558c90f65dc22213c0da9580b33f2d10c35216...,2019-11-08 22:23:58,Movie,Cinemas,Movies or Cinemas,Guest,Unknown,NaN,NaN,50000,4
13732,ca7e910f39fa0c5cfc13ce6da2908c5dc44383cd0e5dfd...,bf005c92cdace85983e5c7e371bdf9cf9fa3f8d51677e0...,2019-11-08 23:02:39,Movie,Cinemas,Movies or Cinemas,Guest,Unknown,NaN,NaN,35000,4
13734,ca7e910f39fa0c5cfc13ce6da2908c5dc44383cd0e5dfd...,faaaa00886a475a0cf89eded054725faff88714c6d087f...,2019-11-08 23:07:19,Movie,Cinemas,Movies or Cinemas,Guest,Unknown,NaN,NaN,35000,4


In [18]:
df['gender'].isna().value_counts()

False    15115
True        23
Name: gender, dtype: int64

In [19]:
df.groupby('user_type')['gender'].value_counts()

user_type   gender
Guest       none      2806
            female       3
Registered  female    6573
            male      5686
            none        47
Name: gender, dtype: int64

In [20]:
df[df['gender'].isna()]['user_type'].value_counts()

Guest    23
Name: user_type, dtype: int64

In [21]:
df[(df['user_type']=='Guest') & (df['gender'].isna())].head()

,user_id,event_id,transaction_date,event_type,organization_type,event_category,user_type,user_location,gender,age_when_register,price,total_ticket_sold
100,a0145331389e0662f4c99950648bad94da75583d354377...,a1de5e094330672b35dd2f89827649bac9f64ecd5777de...,2019-02-23 11:04:25,Movie,Cinemas,Movies or Cinemas,Guest,Unknown,NaN,NaN,45000,2
858,fed7fa12df6ff811b748e624f7177ad8d4ed4c2a75b49e...,7c1be0c107214fce0b35d09b4647b588108aa2d25c6cf7...,2019-04-21 09:49:41,Movie,Cinemas,Movies or Cinemas,Guest,Unknown,NaN,NaN,75000,2
7594,ae72051054c987c1aca286cd33d9081d34f6ba284693ba...,40affd7b6039825eefbddaf2557d5358abfcb2d3e4a72d...,2019-11-21 08:06:15,Movie,Cinemas,Movies or Cinemas,Guest,Unknown,NaN,NaN,35000,1
9773,78310f1b44b7e2edd0e61b0cc54de97e58ab35df437f2a...,74d1b71fa7a7a7c15f33371cdd80d1ed0462a4c21d6965...,2019-04-20 08:33:27,Movie,Cinemas,Movies or Cinemas,Guest,Unknown,NaN,NaN,50000,1
9832,78310f1b44b7e2edd0e61b0cc54de97e58ab35df437f2a...,658866c9cd70c8ad55d34d5313913474227d67d076de7b...,2019-04-21 08:12:38,Movie,Cinemas,Movies or Cinemas,Guest,Unknown,NaN,NaN,50000,1


In [22]:
df['gender'] = df['gender'].fillna('none')

In [23]:
df.isna().sum()

user_id                 0
event_id                0
transaction_date        0
event_type              0
organization_type       0
event_category          0
user_type               0
user_location           0
gender                  0
age_when_register    2882
price                   0
total_ticket_sold       0
dtype: int64

> Now to **age when register**.

In [24]:
df[df['age_when_register'].isna()]['user_type'].value_counts()

Guest         2832
Registered      50
Name: user_type, dtype: int64

In [25]:
df[df['age_when_register'].notna()]['user_type'].value_counts()

Registered    12256
Name: user_type, dtype: int64

> From above information, we can see that every missing values in this category are dominated by the **Guest** users. We will try to explore and mitigate this below. 

In [26]:
df[df['user_type']=='Registered'].groupby('age_when_register')['event_id'].value_counts()

age_when_register  event_id                                                        
9.0                2a97a01dc018b0f22f0ea10980bd9770aed9d336c630b5cd7cbcf7ee509faa0a    2
                   f08be54309c7f86688252774f0626c22977993346451b4e98e67a1cb5e7a5a20    2
                   fe89344cf250678cb14f8ae1a39cc04d95d5d88959bebd2203922841952bb649    2
                   06cd3fac6a8acd2dccad695b512c92dd45fd8c3aa5868a78ecf6d09d147788ce    1
                   07d0425ba9d303c49c176c68d480de8f79229d0bbd573a652c7c5d33d4dca289    1
                                                                                      ..
65.0               69fe75ed7b6985ec63a3c04d38737448cbfbc64da10ae32f5bf3bc5d83ec9b04    1
67.0               a260924adcd2b33b81220b40b616ea3f8f58f53ee8877810a16439381f7050c8    1
68.0               9ef47c758fe7c23fc84dd41885c064ccee99511cb262a04e32380c07db847563    1
70.0               a260924adcd2b33b81220b40b616ea3f8f58f53ee8877810a16439381f7050c8    1
90.0               a5a824d

> Seeing above data, it seems a little bit odd since there are user with and user with 9, 70 and 90 age. I'll try to explore it below.

In [27]:
df['age_when_register'].value_counts().head(20)

49.0    1696
21.0    1365
22.0     914
20.0     863
25.0     718
19.0     705
24.0     689
26.0     660
23.0     658
18.0     562
27.0     475
28.0     385
29.0     308
30.0     225
13.0     214
17.0     206
31.0     166
32.0     147
33.0     136
38.0     101
Name: age_when_register, dtype: int64

In [28]:
df['age_when_register'].value_counts(ascending=True).head(20)

57.0     1
62.0     1
54.0     1
68.0     1
61.0     1
90.0     1
67.0     1
70.0     1
60.0     1
65.0     2
63.0     2
59.0     2
53.0     2
58.0     3
51.0     3
11.0     4
56.0     4
52.0     9
55.0    10
50.0    11
Name: age_when_register, dtype: int64

In [29]:
df['age_when_register'].sort_values(ascending=False).head(15)

50       90.0
9774     70.0
8820     68.0
807      67.0
726      65.0
32       65.0
2382     63.0
2013     63.0
2718     62.0
15089    61.0
2071     60.0
2367     59.0
171      59.0
10297    58.0
11145    58.0
Name: age_when_register, dtype: float64

In [30]:
df['age_when_register'].sort_values(ascending=True).head(15)

2463     9.0
800      9.0
2490     9.0
11243    9.0
406      9.0
10775    9.0
1202     9.0
1941     9.0
10894    9.0
10006    9.0
10005    9.0
2517     9.0
840      9.0
184      9.0
11004    9.0
Name: age_when_register, dtype: float64

In [31]:
df[df['age_when_register']>65]

,user_id,event_id,transaction_date,event_type,organization_type,event_category,user_type,user_location,gender,age_when_register,price,total_ticket_sold
50,b767ab41e1191831cfc628570723d9df9d9027f8e99ad1...,a5a824d4c48633aec0797b247f4f78f31b0026511ea4e2...,2019-02-22 04:03:12,Movie,Cinemas,Movies or Cinemas,Registered,Bandung,male,90.0,40000,2
807,b1687d163b8c7098e963b6016e67687905e6dba44bfb34...,a260924adcd2b33b81220b40b616ea3f8f58f53ee88778...,2019-04-20 08:30:40,Movie,Cinemas,Movies or Cinemas,Registered,Surabaya,female,67.0,50000,1
8820,caa750287dc118fb0af5db8f3e14bf30eb0e317df5bf4a...,9ef47c758fe7c23fc84dd41885c064ccee99511cb262a0...,2020-01-31 10:46:43,Movie,Cinemas,Movies or Cinemas,Registered,Unknown,male,68.0,42500,2
9774,893af0f16a3323bde1174b2befecdd75393d790656a020...,a260924adcd2b33b81220b40b616ea3f8f58f53ee88778...,2019-04-20 09:01:49,Movie,Cinemas,Movies or Cinemas,Registered,Surabaya,male,70.0,50000,1


In [32]:
df[df['age_when_register']<10]

,user_id,event_id,transaction_date,event_type,organization_type,event_category,user_type,user_location,gender,age_when_register,price,total_ticket_sold
111,b23636f78956ef7d3b924619508ec40761217b04a81f89...,2a97a01dc018b0f22f0ea10980bd9770aed9d336c630b5...,2019-02-24 04:07:52,Movie,Cinemas,Movies or Cinemas,Registered,Palembang,male,9.0,35000,2
128,a2728415e009ed304106953af0279da024cdaab423a233...,ae4ad8867966d64952ff5923f3308caa5076494c607cac...,2019-02-24 15:40:21,Movie,Cinemas,Movies or Cinemas,Registered,Purwokerto,female,9.0,30000,2
184,49a2258de669dfb579987f31a6672955e3ebd65a8369bf...,c4f2037e66f080ab370d86022ace29db176fdcf041897f...,2019-02-25 11:36:55,Movie,Cinemas,Movies or Cinemas,Registered,Yogyakarta,male,9.0,30000,2
406,cec853fe1296ed7c1f6d3687b0de5947a2e26aabb793bd...,b609fe1607357e28371b3db4d2473b394d3efe8f5d8f4c...,2019-02-27 12:26:54,Movie,Cinemas,Movies or Cinemas,Registered,Bekasi,male,9.0,30000,2
424,1411aa4324bea632dc0b70bd0d9dae6257eec11b4be8be...,e49694caa9bf4119f97bbb5a9d60c3f22ed1749c7f43f5...,2019-02-27 13:51:56,Movie,Cinemas,Movies or Cinemas,Registered,Palembang,male,9.0,35000,2
...,...,...,...,...,...,...,...,...,...,...,...,...
14805,2e31eb80515999393fd6ebcbe9726d872b7b949bd55281...,ded63d1e197eed5285c268ce13416f65aadcde802819b5...,2019-11-21 11:08:20,Movie,Cinemas,Movies or Cinemas,Registered,Yogyakarta,female,9.0,35000,1
14806,2e31eb80515999393fd6ebcbe9726d872b7b949bd55281...,615c21905d23197d99e21d8211f30821b923b3baf8b5ed...,2019-11-21 11:20:41,Movie,Cinemas,Movies or Cinemas,Registered,Yogyakarta,female,9.0,35000,1
14807,2e31eb80515999393fd6ebcbe9726d872b7b949bd55281...,cd322f8574685973984f6ad1e1192a0ba5e49ce7181937...,2019-11-21 11:22:44,Movie,Cinemas,Movies or Cinemas,Registered,Yogyakarta,female,9.0,35000,1
14809,2e31eb80515999393fd6ebcbe9726d872b7b949bd55281...,99a4090c558c90f65dc22213c0da9580b33f2d10c35216...,2019-11-21 11:30:03,Movie,Cinemas,Movies or Cinemas,Registered,Yogyakarta,female,9.0,50000,1


In [33]:
df.groupby('event_id')['age_when_register'].value_counts()

event_id                                                          age_when_register
000c00e8322eb941989528878f01f6add536a8605fbc0ce7b51d435257950ae0  19.0                 1
001ddd446a41b0fc4578b077b57f7c51faf3fe0d420eaa4ac256f4427c5fb765  21.0                 1
0021a23a1ad6542f63317318470cf33a00d9bb97fc28c5bee08ab94ceb898cb1  33.0                 1
00529ed280a461499121dfe398548b5f8abe8931cd8370f9480a0cc16d3fea51  12.0                 1
0052b33771d32eb1cb5585f030e6ca9b9b937ada33c26786f28258cc48fd8c9b  20.0                 1
                                                                                      ..
ffbdae7028c8079b6f6d9e643cdfc6fbd2b532649f759e6f6186a2a76e1f6fe8  23.0                 1
ffcb2a5a0c81b060ec7718c995054191462b77ad36e0525591a92d6fc289144f  21.0                 1
ffd45f9312cc2c9758c74def3d4181451d750dc3141e5679ff2686eb7acf128d  24.0                 1
fff6b720244784039b5c481df195b63b0def753a5011bdee44f3cf535f5a5220  29.0                 1
fffe648ef4d74ac326dbb995d3

In [34]:
age_isna = df[df['age_when_register'].isna()]
age_notna = df[df['age_when_register'].notna()]

In [35]:
age_notna.groupby('event_id')['age_when_register'].value_counts().head(30)

event_id                                                          age_when_register
000c00e8322eb941989528878f01f6add536a8605fbc0ce7b51d435257950ae0  19.0                 1
001ddd446a41b0fc4578b077b57f7c51faf3fe0d420eaa4ac256f4427c5fb765  21.0                 1
0021a23a1ad6542f63317318470cf33a00d9bb97fc28c5bee08ab94ceb898cb1  33.0                 1
00529ed280a461499121dfe398548b5f8abe8931cd8370f9480a0cc16d3fea51  12.0                 1
0052b33771d32eb1cb5585f030e6ca9b9b937ada33c26786f28258cc48fd8c9b  20.0                 1
00560397c680d37c75926112df54dfb06ee38d08f09ddf055e246ce662f650fc  24.0                 1
0069a295cb6c8436cf51728552bc9bf487a73b6a0ef4b3141b480c419e145631  19.0                 1
                                                                  22.0                 1
00863f9acccb75ecd72389e5f9bfc5096f5d8328a6c207d322781c1704ac1cae  31.0                 1
0092a60331bf5a4903dc9e09f2413fece4e8421ac251ce5caf56c57378ad4fbe  21.0                 1
009632db45228c03be6f143059

In [36]:
age_isna[age_isna['event_id'].isin(age_notna['event_id'])]

,user_id,event_id,transaction_date,event_type,organization_type,event_category,user_type,user_location,gender,age_when_register,price,total_ticket_sold
94,97f0e8938bbfe3e6245fee82693ae69577545ac12c4f90...,97ee1568136dde752d5d6d9f1c64d1e368d3a060246732...,2019-02-23 09:06:19,Movie,Cinemas,Movies or Cinemas,Guest,Unknown,none,NaN,45000,2
103,2143f75c413d717ea7801ae7f5f5c00afbf2fe59ad226c...,89f275a18d3434f69efe7e6d07726199bf8053e312b3bb...,2019-02-23 12:17:43,Movie,Cinemas,Movies or Cinemas,Guest,Unknown,none,NaN,35000,2
124,e79876d65d2e076896edf083a73cc17908372afd55c4f4...,bda11b0424bf6bab9363b1a984d112f7e15a22d4336922...,2019-02-24 11:43:06,Movie,Cinemas,Movies or Cinemas,Guest,Unknown,none,NaN,35000,2
154,4f5ec2fdc796bc665a9714900882b4c379a21f98e95b0e...,31d10df6079307172dac092d0de192cbabe339748153cd...,2019-02-25 05:47:12,Movie,Cinemas,Movies or Cinemas,Guest,Unknown,none,NaN,35000,2
155,b81ec1fc08c1dd6c7db2f655b94ab61b9552d43d49dd1d...,31d10df6079307172dac092d0de192cbabe339748153cd...,2019-02-25 05:49:59,Movie,Cinemas,Movies or Cinemas,Guest,Unknown,none,NaN,35000,2
...,...,...,...,...,...,...,...,...,...,...,...,...
14950,5dd3d1b6c042caaea56f4415b38a7ef8a59ac39b918e14...,99a4090c558c90f65dc22213c0da9580b33f2d10c35216...,2019-11-23 05:48:32,Movie,Cinemas,Movies or Cinemas,Guest,Unknown,none,NaN,50000,1
14951,42c3e23b53b620239bd38e1bca32cb3581ee7c0e2a4633...,41643007db28aa053900e2d6b38b5ae6fd86e67cae9f1a...,2019-11-23 05:58:17,Movie,Cinemas,Movies or Cinemas,Guest,Unknown,none,NaN,35000,1
14956,8fbccc99b8d21dd7966ae60d6c0548671ccbb99aa781cc...,cd322f8574685973984f6ad1e1192a0ba5e49ce7181937...,2019-11-23 06:58:52,Movie,Cinemas,Movies or Cinemas,Guest,Unknown,none,NaN,35000,1
14958,497c731a86ede5e072472b333b62b0e8320b2b7c2c0620...,0aceac5989647d09420a7c7d12691ec40c27cb3813b1bc...,2019-11-23 07:15:04,Movie,Cinemas,Movies or Cinemas,Guest,Unknown,none,NaN,35000,1


In [37]:
age = age_notna[age_notna['event_id'].isin(age_isna['event_id'])]

In [38]:
age['event_id'].value_counts().head(10)

f2e10a84bc034e6d8d518eb96237d1ccce78893fc024127b004041bfe37c1804    143
c71952786040a4b56ac78f3bd22b8608e4b32da54b457fc9aa392f8f598a8552    136
939cc358d7225da272f0cb95bb587893ad86f810751b688aa0c2bde5cd8a1181    134
c41d40719d214a3a52f56c4e8d252317814c0565e6707c570c230f5e4b8ac14e    129
388f0a9896b88bd58dbc24d7060887bc808a3e6743a7a8d4b9bbfb382b3e1d6f    121
d1df2753a4c0408015cbc3d95cc640005104b17b45c7faf7f51ef235519a6171    118
fe88240ce66b31fd90245a0e19268c814b7255e29ce3a6fcfe204ecdc28e6bf0    116
faaaa00886a475a0cf89eded054725faff88714c6d087f41ba616940e5d67ed5    116
bf005c92cdace85983e5c7e371bdf9cf9fa3f8d51677e08a03131d588783089c    113
1532bb3b327ee704d022769c499067b85d516a3a015e43beafd127c3b4e3acea    109
Name: event_id, dtype: int64

In [39]:
# Locating event_id

age_mov1 = age_isna[age_isna['event_id'] == 'f2e10a84bc034e6d8d518eb96237d1ccce78893fc024127b004041bfe37c1804']
age_mov2 = age_isna[age_isna['event_id'] == 'c71952786040a4b56ac78f3bd22b8608e4b32da54b457fc9aa392f8f598a8552']
age_mov3 = age_isna[age_isna['event_id'] == '939cc358d7225da272f0cb95bb587893ad86f810751b688aa0c2bde5cd8a1181']
age_mov4 = age_isna[age_isna['event_id'] == 'c41d40719d214a3a52f56c4e8d252317814c0565e6707c570c230f5e4b8ac14e']
age_mov5 = age_isna[age_isna['event_id'] == '388f0a9896b88bd58dbc24d7060887bc808a3e6743a7a8d4b9bbfb382b3e1d6f']
age_mov6 = age_isna[age_isna['event_id'] == 'd1df2753a4c0408015cbc3d95cc640005104b17b45c7faf7f51ef235519a6171']
age_mov7 = age_isna[age_isna['event_id'] == 'faaaa00886a475a0cf89eded054725faff88714c6d087f41ba616940e5d67ed5']
age_mov8 = age_isna[age_isna['event_id'] == 'fe88240ce66b31fd90245a0e19268c814b7255e29ce3a6fcfe204ecdc28e6bf0']
age_mov9 = age_isna[age_isna['event_id'] == 'bf005c92cdace85983e5c7e371bdf9cf9fa3f8d51677e08a03131d588783089c']
age_mov10 = age_isna[age_isna['event_id'] == '1532bb3b327ee704d022769c499067b85d516a3a015e43beafd127c3b4e3acea']

In [40]:
# Measure filled values

age_fill1 = round(age[age['event_id']=='f2e10a84bc034e6d8d518eb96237d1ccce78893fc024127b004041bfe37c1804']['age_when_register'].mean())
age_fill2 = round(age[age['event_id']=='c71952786040a4b56ac78f3bd22b8608e4b32da54b457fc9aa392f8f598a8552']['age_when_register'].mean())
age_fill3 = round(age[age['event_id']=='939cc358d7225da272f0cb95bb587893ad86f810751b688aa0c2bde5cd8a1181']['age_when_register'].mean())
age_fill4 = round(age[age['event_id']=='c41d40719d214a3a52f56c4e8d252317814c0565e6707c570c230f5e4b8ac14e']['age_when_register'].mean())
age_fill5 = round(age[age['event_id']=='388f0a9896b88bd58dbc24d7060887bc808a3e6743a7a8d4b9bbfb382b3e1d6f']['age_when_register'].mean())
age_fill6 = round(age[age['event_id']=='d1df2753a4c0408015cbc3d95cc640005104b17b45c7faf7f51ef235519a6171']['age_when_register'].mean())
age_fill7 = round(age[age['event_id']=='faaaa00886a475a0cf89eded054725faff88714c6d087f41ba616940e5d67ed5']['age_when_register'].mean())
age_fill8 = round(age[age['event_id']=='fe88240ce66b31fd90245a0e19268c814b7255e29ce3a6fcfe204ecdc28e6bf0']['age_when_register'].mean())
age_fill9 = round(age[age['event_id']=='bf005c92cdace85983e5c7e371bdf9cf9fa3f8d51677e08a03131d588783089c']['age_when_register'].mean())
age_fill10 = round(age[age['event_id']=='1532bb3b327ee704d022769c499067b85d516a3a015e43beafd127c3b4e3acea']['age_when_register'].mean())

In [41]:
# Filling missing value

age_mov1['age_when_register'] = age_mov1['age_when_register'].fillna(age_fill1)
age_mov2['age_when_register'] = age_mov2['age_when_register'].fillna(age_fill2)
age_mov3['age_when_register'] = age_mov3['age_when_register'].fillna(age_fill3)
age_mov4['age_when_register'] = age_mov4['age_when_register'].fillna(age_fill4)
age_mov5['age_when_register'] = age_mov5['age_when_register'].fillna(age_fill5)
age_mov6['age_when_register'] = age_mov6['age_when_register'].fillna(age_fill6)
age_mov7['age_when_register'] = age_mov7['age_when_register'].fillna(age_fill7)
age_mov8['age_when_register'] = age_mov8['age_when_register'].fillna(age_fill8)
age_mov9['age_when_register'] = age_mov9['age_when_register'].fillna(age_fill9)
age_mov10['age_when_register'] = age_mov10['age_when_register'].fillna(age_fill10)

In [42]:
age_filled = pd.concat([age_mov1, age_mov2, age_mov3, age_mov4, age_mov5, age_mov6, age_mov7, age_mov8, age_mov9, age_mov10])

In [43]:
age_isna['age_when_register'] = age_isna['age_when_register'].fillna(age_filled['age_when_register'])

In [44]:
age_fill = pd.concat([age_isna, age_notna])

In [45]:
# Merging to original dataframe

df['age_when_register'] = df['age_when_register'].fillna(age_fill['age_when_register'])

In [46]:
df.isna().sum()

user_id                 0
event_id                0
transaction_date        0
event_type              0
organization_type       0
event_category          0
user_type               0
user_location           0
gender                  0
age_when_register    2363
price                   0
total_ticket_sold       0
dtype: int64

In [48]:
df[df['user_type']=='Guest']['age_when_register'].median()

24.0

In [49]:
df['age_when_register'] = df['age_when_register'].fillna(24)

In [50]:
pd.DataFrame({'dataFeatures' : df.columns, 'dataType' : df.dtypes.values, 
              'null' : [df[i].isna().sum() for i in df.columns],
              'nullPct' : [((df[i].isna().sum()/len(df[i]))*100).round(1) for i in df.columns],
              'Nunique' : [df[i].nunique() for i in df.columns],
              'uniqueSample' : [list(pd.Series(df[i].unique()).sample()) for i in df.columns]}).reset_index(drop = True)

,dataFeatures,dataType,null,nullPct,Nunique,uniqueSample
0,user_id,object,0,0.0,5038,[819ef1a9ae7999adedfb1ea4d75fd1d920b07e703bc0c...
1,event_id,object,0,0.0,5289,[1a52d32d603962e8288a40f072b9cd34b92e1508bdec5...
2,transaction_date,object,0,0.0,14945,[2019-07-15 05:58:55]
3,event_type,object,0,0.0,1,[Movie]
4,organization_type,object,0,0.0,1,[Cinemas]
5,event_category,object,0,0.0,1,[Movies or Cinemas]
6,user_type,object,0,0.0,2,[Guest]
7,user_location,object,0,0.0,31,[Mojokerto]
8,gender,object,0,0.0,3,[female]
9,age_when_register,float64,0,0.0,60,[41.0]


In [51]:
df.to_csv('dataset\movie_clean.csv', index=False)